### sklearn.ensemble.GradientBoostingClassifier
* class sklearn.ensemble.GradientBoostingClassifier(*, loss='log_loss', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, init=None, random_state=None, max_features=None, verbose=0, max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)
> https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

In [8]:
import pandas as pd

feature_name_df = pd.read_csv('./Human_activity/features.txt', sep='\s+', header=None, names=['column_index', 'column_name'])
feature_name = feature_name_df.iloc[:, 1].values.tolist()
feature_dup_df = feature_name_df.groupby('column_name').count()

def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(), columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x : x[0] + '_' + str(x[1]) if x[1]>0 else x[0], axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df
    
def get_human_dataset():
    # 각 데이터 파일들은 공백으로 분리되어 있으므로 read_csv에서 공백 문자를 sep으로 할당
    feature_name_df = pd.read_csv('./Human_activity/features.txt', sep='\s+', header=None, names=['column_index', 'column_name'])

    # 중복된 피처명을 수정하는 get_new_feature_df()를 이용, 신규 피처명 DataFrame 생성
    new_feature_name_df = get_new_feature_name_df(feature_name_df)

    # DataFrame에 피처명을 컬럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = new_feature_name_df.iloc[:, 1].values.tolist()

    # 학습 피처 데이터 셋과 테스트 피처 데이터를 DataFrame으로 로딩, 컬럼명은 feature_name 적용
    X_train = pd.read_csv('./Human_activity/train/X_train.txt', sep='\s+', names=feature_name)
    X_test = pd.read_csv('./Human_activity/test/X_test.txt', sep='\s+', names=feature_name)

    # 학습 레이블과 테스트 레이블 데이터를 DataFrame으로 로딩, 컬럼명은 action으로 부여
    y_train = pd.read_csv('./Human_activity/train/y_train.txt', sep='\s+', header=None, names=['action'])
    y_test = pd.read_csv('./Human_activity/test/y_test.txt', sep='\s+', header=None, names=['action'])

    # 로드된 학습/테스트용 DataFrame을 모두 반환
    return X_train, X_test, y_train, y_test

In [9]:
from sklearn.ensemble import GradientBoostingClassifier
import time
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = get_human_dataset()

# GBM 수행 시간 측정을 위함. 시작 시간 설정.
start_time = time.time()

gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train, y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)

print(f'GBM 정확도 : {gb_accuracy}')
print(f'GBM 수행 시간 : {time.time() - start_time} 초')

c:\Users\YS\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GBM 정확도 : 0.9389209365456397
GBM 수행 시간 : 1217.9504833221436 초


20분 걸리는 거 실화인지?

### xgboost
* class xgboost.XGBRegressor(*, objective='reg:squarederror', **kwargs)
> https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn

In [15]:
import xgboost as xgb
# 반드시 튜닝해야할 파라미터는  min_child_weight / max_depth / gamma

clf=xgb.XGBClassifier()   # 파라미터 넣어줌.   모델생성
clf.fit()                 # 파라미터 넣어줌.   데이터학습.                      
clf.coef_                 # gbtree는 볼수없음.
evals_result=clf.evals_result()
clf.feature_importances_
clf.predict()                 #파라미터 넣어줌.  예측

xgb.XGBClassifier(
    # General Parameter
    booster='gbtree' # 트리,회귀(gblinear) 트리가 항상 
                     # 더 좋은 성능을 내기 때문에 수정할 필요없다고한다.
    silent=True  # running message출력안한다.
                 # 모델이 적합되는 과정을 이해하기위해선 False으로한다.
    min_child_weight=10   # 값이 높아지면 under-fitting 되는 
                          # 경우가 있다. CV를 통해 튜닝되어야 한다.
    max_depth=8     # 트리의 최대 깊이를 정의함. 
                    # 루트에서 가장 긴 노드의 거리.
                    # 8이면 중요변수에서 결론까지 변수가 9개거친다.
                    # Typical Value는 3-10. 
    gamma =0    # 노드가 split 되기 위한 loss function의 값이
                # 감소하는 최소값을 정의한다. gamma 값이 높아질 수록 
                # 알고리즘은 보수적으로 변하고, loss function의 정의
                #에 따라 적정값이 달라지기때문에 반드시 튜닝.
    nthread =4    # XGBoost를 실행하기 위한 병렬처리(쓰레드)
                  #갯수. 'n_jobs' 를 사용해라.
    colsample_bytree=0.8   # 트리를 생성할때 훈련 데이터에서 
                           # 변수를 샘플링해주는 비율. 보통0.6~0.9
    colsample_bylevel=0.9  # 트리의 레벨별로 훈련 데이터의 
                           #변수를 샘플링해주는 비율. 보통0.6~0.9
    n_estimators =(int)   #부스트트리의 양
                          # 트리의 갯수. 
    objective = 'reg:linear','binary:logistic','multi:softmax',
                'multi:softprob'  # 4가지 존재.
            # 회귀 경우 'reg', binary분류의 경우 'binary',
            # 다중분류경우 'multi'- 분류된 class를 return하는 경우 'softmax'
            # 각 class에 속할 확률을 return하는 경우 'softprob'
    random_state =  # random number seed.
                    # seed 와 동일.
)


XGBClassifier.fit(
    X (array_like)     # Feature matrix ( 독립변수)
                       # X_train
    Y (array)          # Labels (종속변수)
                       # Y_train
    eval_set           # 빨리 끝나기 위해 검증데이터와 같이써야한다.  
                       # =[(X_train,Y_train),(X_vld, Y_vld)]
    eval_metric = 'rmse','error','mae','logloss','merror',
                'mlogloss','auc'  
              # validation set (검증데이터)에 적용되는 모델 선택 기준.
              # 평가측정. 
              # 회귀 경우 rmse ,  분류 -error   이외의 옵션은 함수정의
    early_stopping_rounds=100,20
              # 100번,20번 반복동안 최대화 되지 않으면 stop
)
[출처] 파이썬 Scikit-Learn형식 XGBoost 파라미터|작성자 현무
https://blog.naver.com/PostView.nhn?blogId=gustn3964&logNo=221431714122

SyntaxError: invalid syntax. Perhaps you forgot a comma? (177557653.py, line 13)

In [11]:
from xgboost import XGBClassifier

# 모델 선언 예시
model = XGBClassifier(n_estimators=500, learning_rate=0.2, max_depth=4, random_state = 32)

model.fit(X_train, y_train)
model.predict(X_test)
#y_pred = model.predict(X_test) # 예측 라벨(0과 1로 예측)

# 예측 라벨과 실제 라벨 사이의 정확도 측정
#accuracy_score(y_pred, y_test) # 0.7847533632286996

# model=XGBClassifier(booster='gbtree', 
#                     colsample_bylevel=0.9, 
#                     colsample_bytree=0.8, 
#                     gamma=0, 
#                     max_depth=8, 
#                     min_child_weight=3, 
#                     n_estimators=50, 
#                     nthread=4, 
#                     objective='binary:logistic', 
#                     random_state=2, 
#                     silent= True)
# model.fit(train_X,train_Y, eval_set=[(val_X,val_Y)],
#              early_stopping_rounds=50,verbos=5)
# model.predict(test_X)

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3 4 5], got [1 2 3 4 5 6]

In [25]:
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)
pd.Series(y_train)
pd.Series(y_test)

X_train, X_test, y_train, y_test = get_human_dataset()
start_time = time.time()
xg_clf = XGBClassifier(n_estimators=500, learning_rate=0.2, max_depth=4, random_state = 32)
xg_clf.fit(X_train, y_train)
xg_pred = xg_clf.predict(X_test)
xg_accuracy = accuracy_score(y_test, xg_pred)
print('XG 정확도: {0: .4f}'.format(xg_accuracy))
print('XG 수행 시간: {0: .1f} 초'.format(time.time() - start_time))

c:\Users\YS\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3 4 5], got [1 2 3 4 5 6]

In [26]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)
pd.Series(y_train)
pd.Series(y_test)

model = XGBClassifier(n_estimators=500, learning_rate=0.2, max_depth=4, random_state = 32)
model.fit(X_train, y_train)

y_pred = model.predict(X_test) # 예측 라벨(0과 1로 예측)
# 예측 라벨과 실제 라벨 사이의 정확도 측정
accuracy_score(y_pred, y_test)

c:\Users\YS\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9463861554122837